Perform the NLP on DATA and perform the model trainnig..

In [28]:
# import basic libeary
import pandas as pd
import numpy as np

In [29]:
# load the dataset
df=pd.read_json('reviews_Office_Products_5.json.gz',lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53258 entries, 0 to 53257
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      53258 non-null  object
 1   asin            53258 non-null  object
 2   reviewerName    52799 non-null  object
 3   helpful         53258 non-null  object
 4   reviewText      53258 non-null  object
 5   overall         53258 non-null  int64 
 6   summary         53258 non-null  object
 7   unixReviewTime  53258 non-null  int64 
 8   reviewTime      53258 non-null  object
dtypes: int64(2), object(7)
memory usage: 3.7+ MB


In [31]:
# choice the important columns
df=df[['overall','reviewText']]
df.head()

,overall,reviewText
0,5,"I bought my first HP12C in about 1984 or so, a..."
1,5,WHY THIS BELATED REVIEW? I feel very obliged t...
2,2,I have an HP 48GX that has been kicking for mo...
3,5,I've started doing more finance stuff recently...
4,5,For simple calculations and discounted cash fl...


In [32]:
df.shape

(53258, 2)

In [33]:
# Find the misssing values
df.isna().sum()

overall       0
reviewText    0
dtype: int64

In [34]:
# Output unique values
df['overall'].unique()

array([5, 2, 4, 3, 1])

In [35]:
# Check the data is imbalanced or not
df['overall'].value_counts()

overall
5    30327
4    15015
3     5060
2     1726
1     1130
Name: count, dtype: int64

In [36]:
# Covert the data into postive and negative revivew
# Use the lambda function
df['overall']=df['overall'].apply(lambda x:0 if x<4 else 1)


In [37]:
# check the alanced of dataset
df['overall'].value_counts(normalize=True)*100

overall
1    85.136505
0    14.863495
Name: proportion, dtype: float64

Conclusion:- Data was imbalanced

# Cleaning: lowercase, remove stopwords, punctuation, etc.


In [38]:
# Lower the dataset
df['reviewText']=df['reviewText'].str.lower()
df['reviewText']

0        i bought my first hp12c in about 1984 or so, a...
1        why this belated review? i feel very obliged t...
2        i have an hp 48gx that has been kicking for mo...
3        i've started doing more finance stuff recently...
4        for simple calculations and discounted cash fl...
                               ...                        
53253    what i like about this scale is you can power ...
53254    this accuteck shippro digital scale works very...
53255    i ship a lot of stuff.  i sell small parts, ma...
53256    this is a great little scale to have. it can w...
53257    when asked to review this scale, i almost decl...
Name: reviewText, Length: 53258, dtype: object

### Cleaning the dataset


Data size are big so it take time to perform the cleaning

In [39]:
# pip install swifter

In [40]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from bs4 import BeautifulSoup
import swifter


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sriva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
# if we have the large data set so use the function that are cleaning function
def cleaning_dataset(text):
    text=re.sub('[^a-z A-Z 0-9]',' ',text)
    # remove the url
    text=re.sub(r'http\S+|www\S+|https\S+', '', text)
    # remove the HTML
    text=BeautifulSoup(text, 'html.parser').get_text()
    # lower the dataset
    text=text.lower()
    # remove the stopwords
    words=[word for word in text.split() if word not in  set(stopwords.words('english'))]
    return ' '.join(words)

In [42]:
# Call the cleaning dataset function
df['reviewText']=df['reviewText'].swifter.apply(cleaning_dataset)

Pandas Apply:   0%|          | 0/53258 [00:00<?, ?it/s]

In [43]:
df.to_csv('Clean_reviews_Office_Products.csv',index=False)

In [44]:
# Load the Cleandataset
data=pd.read_csv('Clean_reviews_Office_Products.csv')

In [45]:
data.head()

,overall,reviewText
0,1,bought first hp12c 1984 served faithfully 2002...
1,1,belated review feel obliged share views old wo...
2,0,hp 48gx kicking twenty years hp 11 25 years ol...
3,1,started finance stuff recently went looking go...
4,1,simple calculations discounted cash flows one ...


In [46]:
# Apply wordlemmalizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

def lemmatizer_word(text):
    text=str(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# call the function
data['reviewText']=data['reviewText'].swifter.apply(lambda x:lemmatizer_word(x))

Pandas Apply:   0%|          | 0/53258 [00:00<?, ?it/s]

### perform word2vec

In [47]:
import gensim
import gensim.downloader as api
wv=api.load('word2vec-google-news-300')
import nltk
from nltk import word_tokenize

In [48]:
# Perform the word_token
token=word_tokenize
data['reviewText']=data['reviewText'].apply(token)

In [49]:
data.head()

,overall,reviewText
0,1,"[bought, first, hp12c, 1984, served, faithfull..."
1,1,"[belated, review, feel, obliged, share, view, ..."
2,0,"[hp, 48gx, kicking, twenty, year, hp, 11, 25, ..."
3,1,"[started, finance, stuff, recently, went, look..."
4,1,"[simple, calculation, discounted, cash, flow, ..."


In [50]:
# Apply word2vec
from gensim.models import Word2Vec
model_w2v=Word2Vec(sentences=data['reviewText'])
print("Vocabulary size:", len(model_w2v.wv))

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


Vocabulary size: 14959


In [51]:
# Apply average word2vec
import numpy as np
def get_avg_vector(token,model,vector):
    vectors=[model.wv[word] for word in token if word in model.wv]

    # If no word found in vocab return a zero vector
    if len(vectors)==0:
        return np.zeros(vector)
    return np.mean(vectors,axis=0)
    

In [ ]:
data['reviewText'] = data['reviewText'].apply(lambda x: get_avg_vector(x, model_w2v, 100))

In [55]:
X=data['reviewText']
X

0        [-0.16485932, -0.30395684, -0.14812472, 0.0847...
1        [0.14638364, -0.08561607, -0.1792544, -0.18312...
2        [-0.042063203, 0.172079, -0.007007351, 0.05918...
3        [0.064247884, 0.099953316, -0.08769544, -0.152...
4        [0.110752665, 0.09924755, -0.03874199, -0.0972...
                               ...                        
53253    [-0.393987, 0.047004588, 0.025111325, -0.26687...
53254    [-0.18544377, -0.07720595, -0.004692665, -0.05...
53255    [-0.32110387, -0.34292647, -0.49355736, 0.2469...
53256    [-0.252142, -0.33412036, -0.1602429, 0.2723093...
53257    [-0.3082513, -0.18487701, -0.2665929, 0.004398...
Name: reviewText, Length: 53258, dtype: object

In [56]:
X=np.vstack(X)
y=data['overall']

In [57]:
X.shape,y.shape

((53258, 100), (53258,))

In [73]:
# Handle imbalanced data set
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X, y)


In [74]:
y_res.value_counts()
len(X_res)

90684

### Split the dataset

In [63]:
# independaent feature are X and depedent feature are y

In [75]:
# train to split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_res,y_res,test_size=0.30,random_state=42)

In [76]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((63478, 100), (27206, 100), (63478,), (27206,))

In [69]:
# train the model
!pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/72.0 MB 4.1 MB/s eta 0:00:18
    --------------------------------------- 1.6/72.0 MB 3.8 MB/s eta 0:00:19
   - -------------------------------------- 3.4/72.0 MB 5.5 MB/s eta 0:00:13
   -- ------------------------------------- 4.5/72.0 MB 5.3 MB/s eta 0:00:13
   --- ------------------------------------ 5.5/72.0 MB 5.8 MB/s eta 0:00:12
   --- ------------------------------------ 6.8/72.0 MB 5.5 MB/s eta 0:00:12
   ---- ----------------------------------- 7.6/72.0 MB 5.2 MB/s eta 0:00:13
   ---- ----------------------------------- 8.7/72.0 MB 5.3 MB/s eta 0:00:13
   ----- ---------------------------------- 9.2/72.0 MB 5.2 MB/s eta 0:00:13
   ----- ---------------------------------- 10.2/72.0 MB 5.0 MB/s eta 0:00:13
   ----- ---------------------------------- 10.7/72.0 MB 4.9 MB/s eta 0:00:13
   ------ -

In [79]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize XGBoost model
xgb_model = XGBClassifier(
    n_estimators=500,       # number of trees
    learning_rate=0.1,      # step size
    max_depth=6,            # tree depth
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'   # for binary classification
)

# Train model
xgb_model.fit(X_train, y_train)

# Predict
y_pred = xgb_model.predict(X_test)


c:\Users\sriva\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [00:50:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [80]:
# Accuracy
print("✅ Accuracy:", accuracy_score(y_test, y_pred))

# Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


✅ Accuracy: 0.9310446225097405

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93     13626
           1       0.98      0.88      0.93     13580

    accuracy                           0.93     27206
   macro avg       0.93      0.93      0.93     27206
weighted avg       0.93      0.93      0.93     27206


Confusion Matrix:
 [[13330   296]
 [ 1580 12000]]
